In [1]:
import torch
import torchvision
from torchvision import transforms, datasets
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
import cv2
import numpy as np
from tqdm import tqdm
from collections import OrderedDict
from torch.optim.lr_scheduler import StepLR


In [2]:
train = datasets.MNIST("../data/", train=True, download=True, transform = transforms.Compose([transforms.ToTensor()]))
test = datasets.MNIST("../data/", train=False, download=True, transform = transforms.Compose([transforms.ToTensor()]))

In [3]:
IM_SIZE = 28 * 28
IM_SHAPE = (28, 28)
CPU_training = False
BATCH = 100
use_cuda =  torch.cuda.is_available() and not CPU_training
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

trainset = torch.utils.data.DataLoader(train, batch_size=BATCH, shuffle=True, **kwargs)
testset = torch.utils.data.DataLoader(test, batch_size=BATCH, shuffle=True, **kwargs)

In [24]:
PCA_dim = 2
PCA_2d = nn.Sequential(OrderedDict([
            ('fc1_en', nn.Linear(IM_SIZE, PCA_dim)),
            ('relu1', nn.PReLU()),
            ('fc1_de', nn.Linear(PCA_dim, IM_SIZE)),
            ('sigmoid1', nn.Softmax(dim=0))    
         ])).to(device)


In [ ]:
net = PCA_2d.train()

optimizer = optim.Adadelta(net.parameters(), lr=1.0, rho=0.9, eps=1e-06, weight_decay=0)
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)
EPOCHS = 25

for epoch in range(EPOCHS):
    for X, _ in tqdm(trainset):
        X = X.view(-1, IM_SIZE).to(device)
        net.zero_grad()
        output = net(X)
        loss = F.binary_cross_entropy(output, X)
        loss.backward()
        optimizer.step()
    scheduler.step()
    print(epoch, loss)

 95%|███████████████████████████████████████████████████████████████████████████▏   | 571/600 [00:05<00:00, 123.30it/s]

In [ ]:
"""
SOURCE: https://discuss.pytorch.org/t/how-can-l-load-my-best-model-as-a-feature-extractor-evaluator/17254/6
"""

activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

PCA_2d.fc1_en.register_forward_hook(get_activation('fc1_en'))
PCA_2d.relu1.register_forward_hook(get_activation('relu1'))

In [ ]:
net = PCA_2d.eval()

comp_data = np.empty((10000, PCA_dim))
comp_label = np.empty((10000,))
loss = 0
with torch.no_grad():
    for i, (X, y) in tqdm(enumerate(testset)):
        X = X.view(-1, IM_SIZE).to(device)
        output = net(X)
        comp_data[BATCH * i: BATCH * (i + 1)] = activation['relu1'].cpu()
        comp_label[BATCH * i: BATCH * (i + 1)] = y

        loss += F.mse_loss(output, X, reduction='sum')

print(loss)
if use_cuda:
    X = X.cpu()
    output = output.cpu()
plt.imshow(X[0].view(IM_SHAPE))
plt.show()
plt.imshow(output[0].view(IM_SHAPE))
plt.show()



In [ ]:
plt.scatter(comp_data[:,0], comp_data[:,1], c=comp_label, cmap=plt.cm.rainbow)
plt.show()
